In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

/home/kchen/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/kchen/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
preds = pd.read_csv('../preds/preds_InceptionV3_radimagenet_all_cutout.csv')
preds.head()

,file_path,y_true,y_pred
0,../data/split_1/train/0/104_new.jpg,0.0,0.593562
1,../data/split_1/train/0/104_old.jpg,0.0,0.396177
2,../data/split_1/train/0/107_old.jpg,0.0,0.498852
3,../data/split_1/train/0/108_old.jpg,0.0,0.499771
4,../data/split_1/train/0/109_old.jpg,0.0,0.504240


In [5]:
preds['y_pred'].describe()

count    821.000000
mean       0.537606
std        0.116871
min        0.212202
25%        0.467364
50%        0.553898
75%        0.610304
max        0.914310
Name: y_pred, dtype: float64

In [6]:
preds.shape

(821, 3)

In [7]:
roc_auc_score(preds['y_true'], preds['y_pred'])

0.6786899875066927

In [8]:
test_preds = preds.loc[preds['file_path'].str.contains('test')]
roc_auc_score(test_preds['y_true'], test_preds['y_pred'])

0.7151162790697674

In [9]:
test_preds = preds.loc[preds['file_path'].str.contains('train')]
roc_auc_score(test_preds['y_true'], test_preds['y_pred'])

0.6867421983701053

In [10]:
test_preds = preds.loc[preds['file_path'].str.contains('val')]
roc_auc_score(test_preds['y_true'], test_preds['y_pred'])

0.6196581196581196

In [11]:
old_preds = preds.loc[~preds['file_path'].str.contains('new')]
roc_auc_score(old_preds['y_true'], old_preds['y_pred'])

0.6659845835982199

In [12]:
new_preds = preds.loc[preds['file_path'].str.contains('new')]
roc_auc_score(new_preds['y_true'], new_preds['y_pred'])

0.6708277784510968

In [13]:
#read in the clinical info
clin = pd.read_csv('../data/new_clin_clean.csv')
clin.head()

,study_id,age,race/ethnicity,bmi,density,tumor_grade,tumor_size,tumor_type,margin
0,573,68.0,Non-Hispanic White,26.05,C,2.0,5.0,ILC,0.0
1,95,59.0,Non-Hispanic Black,32.58,A,3.0,15.0,IDC,1.0
2,748,44.0,Non-Hispanic White,25.39,C,3.0,8.0,IDC,1.0
3,391,52.0,Non-Hispanic Black,40.57,B,2.0,10.0,IDC,1.0
4,79,51.0,Non-Hispanic White,32.46,A,1.0,12.0,IDC,1.0


In [14]:
preds_new = preds.loc[preds['file_path'].str.contains('_new')]
preds_new.head()

,file_path,y_true,y_pred
0,../data/split_1/train/0/104_new.jpg,0.0,0.593562
6,../data/split_1/train/0/110_new.jpg,0.0,0.692720
9,../data/split_1/train/0/122_new.jpg,0.0,0.455709
15,../data/split_1/train/0/138_new.jpg,0.0,0.527954
18,../data/split_1/train/0/145_new.jpg,0.0,0.546358


In [15]:
preds_new.shape

(371, 3)

In [16]:
file_paths = preds_new['file_path'].tolist()
file_paths1 = [x.split('/')[-1] for x in file_paths]
study_ids = [x.split('_')[0] for x in file_paths1]
preds_new['study_id'] = study_ids
preds_new.head()

/tmp/ipykernel_268257/2348572271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preds_new['study_id'] = study_ids


,file_path,y_true,y_pred,study_id
0,../data/split_1/train/0/104_new.jpg,0.0,0.593562,104
6,../data/split_1/train/0/110_new.jpg,0.0,0.692720,110
9,../data/split_1/train/0/122_new.jpg,0.0,0.455709,122
15,../data/split_1/train/0/138_new.jpg,0.0,0.527954,138
18,../data/split_1/train/0/145_new.jpg,0.0,0.546358,145


In [17]:
test_study_ids = preds_new['study_id'].tolist()
test_study_ids = [int(x) for x in test_study_ids]
test_clin = clin.loc[clin['study_id'].isin(test_study_ids)]
print(test_clin.shape)
test_clin.head()

(371, 9)


,study_id,age,race/ethnicity,bmi,density,tumor_grade,tumor_size,tumor_type,margin
0,573,68.0,Non-Hispanic White,26.05,C,2.0,5.0,ILC,0.0
1,95,59.0,Non-Hispanic Black,32.58,A,3.0,15.0,IDC,1.0
2,748,44.0,Non-Hispanic White,25.39,C,3.0,8.0,IDC,1.0
3,391,52.0,Non-Hispanic Black,40.57,B,2.0,10.0,IDC,1.0
4,79,51.0,Non-Hispanic White,32.46,A,1.0,12.0,IDC,1.0


In [18]:
test_clin['study_id'] = test_clin['study_id'].astype(int)
test_clin.sort_values(by=['study_id'], inplace=True)
test_clin.reset_index(drop=True, inplace=True)
test_clin.head()

,study_id,age,race/ethnicity,bmi,density,tumor_grade,tumor_size,tumor_type,margin
0,1,46.0,Asian,18.97,D,2.0,10.000000,IDC,1.0
1,2,47.0,Hispanic,40.28,B,2.0,4.000000,IDC,1.0
2,3,61.0,Non-Hispanic White,24.44,C,2.0,7.553571,DCIS,1.0
3,4,59.0,Non-Hispanic White,24.21,C,1.0,10.000000,IDC,1.0
4,5,40.0,Non-Hispanic White,21.80,C,3.0,11.000000,IDC,0.0


In [19]:
preds_new['study_id'] = preds_new['study_id'].astype(int)
preds_new.sort_values(by=['study_id'], inplace=True)
preds_new.reset_index(drop=True, inplace=True)
preds_new.head()

/tmp/ipykernel_268257/2152819623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preds_new['study_id'] = preds_new['study_id'].astype(int)
/home/kchen/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,file_path,y_true,y_pred,study_id
0,../data/split_1/train/1/1_new.jpg,1.0,0.309868,1
1,../data/split_1/train/1/2_new.jpg,1.0,0.794507,2
2,../data/split_1/train/1/3_new.jpg,1.0,0.505771,3
3,../data/split_1/train/1/4_new.jpg,1.0,0.840647,4
4,../data/split_1/train/0/5_new.jpg,0.0,0.339217,5


In [20]:
roc_auc_score(preds_new['y_true'], preds_new['y_pred'])

0.6708277784510968

In [21]:
test_clin['y_pred'] = np.NaN
test_clin.update(preds_new)
test_clin.head()

,study_id,age,race/ethnicity,bmi,density,tumor_grade,tumor_size,tumor_type,margin,y_pred
0,1,46.0,Asian,18.97,D,2.0,10.000000,IDC,1.0,0.309868
1,2,47.0,Hispanic,40.28,B,2.0,4.000000,IDC,1.0,0.794507
2,3,61.0,Non-Hispanic White,24.44,C,2.0,7.553571,DCIS,1.0,0.505771
3,4,59.0,Non-Hispanic White,24.21,C,1.0,10.000000,IDC,1.0,0.840647
4,5,40.0,Non-Hispanic White,21.80,C,3.0,11.000000,IDC,0.0,0.339217


In [22]:
roc_auc_score(test_clin['margin'], test_clin['y_pred'])

0.6708277784510968

In [23]:
test_clin.set_index('study_id', inplace=True)
test_clin.head()

,age,race/ethnicity,bmi,density,tumor_grade,tumor_size,tumor_type,margin,y_pred
study_id,,,,,,,,,
1,46.0,Asian,18.97,D,2.0,10.000000,IDC,1.0,0.309868
2,47.0,Hispanic,40.28,B,2.0,4.000000,IDC,1.0,0.794507
3,61.0,Non-Hispanic White,24.44,C,2.0,7.553571,DCIS,1.0,0.505771
4,59.0,Non-Hispanic White,24.21,C,1.0,10.000000,IDC,1.0,0.840647
5,40.0,Non-Hispanic White,21.80,C,3.0,11.000000,IDC,0.0,0.339217


In [24]:
roc_auc_score(test_clin['margin'], test_clin['y_pred'])

0.6708277784510968

In [25]:
cat_cols = ['race/ethnicity', 'density', 'tumor_grade', 'tumor_type']
for col in cat_cols:
    test_clin[col] = test_clin[col].astype('category')

In [26]:
densities = test_clin['density'].cat.categories.tolist()
race_eths = test_clin['race/ethnicity'].cat.categories.tolist()
tumor_grades = test_clin['tumor_grade'].cat.categories.tolist()
tumor_types = test_clin['tumor_type'].cat.categories.tolist()

In [27]:
#make a dataframe with auroc, sensitivity, specificity, positive predictive value, negative predictive value
res_df = pd.DataFrame(columns = ['subset', 'AUROC', 'AUPRC', 'Sensitivity', 'Specificity', 'PPV', 'NPV'])
for density in densities:
    subset = test_clin.loc[test_clin['density'] == density]
    y_true = subset['margin']
    y_pred = subset['y_pred']
    auroc = roc_auc_score(y_true, y_pred)
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    auprc = auc(recall, precision)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred.round()).ravel()
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    res_df = res_df.append({'subset': density, 'AUROC': auroc, 'AUPRC': auprc, 'Sensitivity': sensitivity, 'Specificity': specificity, 'PPV': ppv, 'NPV': npv}, ignore_index=True)
res_df.head()

,subset,AUROC,AUPRC,Sensitivity,Specificity,PPV,NPV
0,A,0.772727,0.891216,0.909091,0.000000,0.625000,0.000000
1,B,0.667408,0.754888,0.915888,0.238095,0.671233,0.625000
2,C,0.699094,0.729256,0.828947,0.393443,0.630000,0.648649
3,D,0.544061,0.696288,0.724138,0.388889,0.656250,0.466667


In [28]:
res_df.to_csv('../results/res_density.csv', index=False)

In [29]:
pd.crosstab(test_clin['margin'], test_clin['race/ethnicity'])

race/ethnicity,Asian,Hispanic,Non-Hispanic Black,Non-Hispanic White,Other/Unknown
margin,,,,,
0.0,7,9,25,104,3
1.0,5,20,42,156,0


In [30]:
race_eths.remove('Other/Unknown')

In [31]:
#make a dataframe with auroc, sensitivity, specificity, positive predictive value, negative predictive value
res_df = pd.DataFrame(columns = ['subset', 'AUROC', 'AUPRC', 'Sensitivity', 'Specificity', 'PPV', 'NPV'])
for density in race_eths:
    subset = test_clin.loc[test_clin['race/ethnicity'] == density]
    y_true = subset['margin']
    y_pred = subset['y_pred']
    auroc = roc_auc_score(y_true, y_pred)
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    auprc = auc(recall, precision)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred.round()).ravel()
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    res_df = res_df.append({'subset': density, 'AUROC': auroc, 'AUPRC': auprc, 'Sensitivity': sensitivity, 'Specificity': specificity, 'PPV': ppv, 'NPV': npv}, ignore_index=True)
res_df.head()

,subset,AUROC,AUPRC,Sensitivity,Specificity,PPV,NPV
0,Asian,0.771429,0.833030,0.800000,0.285714,0.444444,0.666667
1,Hispanic,0.705556,0.852556,0.950000,0.444444,0.791667,0.800000
2,Non-Hispanic Black,0.753333,0.857587,0.880952,0.440000,0.725490,0.687500
3,Non-Hispanic White,0.629006,0.700432,0.846154,0.269231,0.634615,0.538462


In [32]:
res_df.to_csv('../results/res_raceth.csv', index=False)

In [33]:
test_clin['Race/Ethnicity'] = np.NaN
test_clin.loc[test_clin['race/ethnicity'] == 'Non-Hispanic White', 'Race/Ethnicity'] = 'Non-Hispanic White'
test_clin['Race/Ethnicity'].fillna('Non-White', inplace=True)
test_clin['Race/Ethnicity'].value_counts(dropna=False)

Non-Hispanic White    260
Non-White             111
Name: Race/Ethnicity, dtype: int64

In [34]:
raceth_density = pd.crosstab(test_clin['density'], test_clin['Race/Ethnicity'])
raceth_density


Race/Ethnicity,Non-Hispanic White,Non-White
density,,
A,9,8
B,121,49
C,93,44
D,37,10


In [35]:
raceth_density_norm = pd.crosstab(test_clin['density'], test_clin['Race/Ethnicity'], normalize='columns')
raceth_density_norm

Race/Ethnicity,Non-Hispanic White,Non-White
density,,
A,0.034615,0.072072
B,0.465385,0.441441
C,0.357692,0.396396
D,0.142308,0.090090


In [36]:
raceth_density.to_csv('../results/raceth_density.csv')